In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

24440


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
24435,2021-06-14,47,沖縄県,54,19562,4,163
24436,2021-06-15,47,沖縄県,107,19669,0,163
24437,2021-06-16,47,沖縄県,115,19784,0,163
24438,2021-06-17,47,沖縄県,97,19881,4,167
24439,2021-06-18,47,沖縄県,86,19967,0,167


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11440,8989,147
43,香川県,19240,2081,29
44,高知県,20280,1705,23
45,鳥取県,16120,466,2
46,鹿児島県,23920,3630,33


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
24433,2021-06-12,47,沖縄県,157,19404,3,159,1088.0
24434,2021-06-13,47,沖縄県,104,19508,0,159,1009.0
24435,2021-06-14,47,沖縄県,54,19562,4,163,959.0
24436,2021-06-15,47,沖縄県,107,19669,0,163,907.0
24437,2021-06-16,47,沖縄県,115,19784,0,163,848.0
24438,2021-06-17,47,沖縄県,97,19881,4,167,779.0
24439,2021-06-18,47,沖縄県,86,19967,0,167,720.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref["新規感染者数先週比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change(periods=7)

pref.tail(10)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
24430,2021-06-09,47,沖縄県,174,18936,2,156,1372.0,0.094340,-0.030976,-0.414141
24431,2021-06-10,47,沖縄県,166,19102,0,156,1294.0,-0.045977,-0.012052,-0.319672
24432,2021-06-11,47,沖縄県,145,19247,0,156,1192.0,-0.126506,-0.031880,-0.412955
24433,2021-06-12,47,沖縄県,157,19404,3,159,1088.0,0.082759,-0.028155,-0.398467
24434,2021-06-13,47,沖縄県,104,19508,0,159,1009.0,-0.337580,-0.033687,-0.431694
24435,2021-06-14,47,沖縄県,54,19562,4,163,959.0,-0.480769,-0.040698,-0.480769
24436,2021-06-15,47,沖縄県,107,19669,0,163,907.0,0.981481,0.023964,-0.327044
24437,2021-06-16,47,沖縄県,115,19784,0,163,848.0,0.074766,0.021168,-0.339080
24438,2021-06-17,47,沖縄県,97,19881,4,167,779.0,-0.156522,0.005376,-0.415663
24439,2021-06-18,47,沖縄県,86,19967,0,167,720.0,-0.113402,0.007248,-0.406897


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["日付","都道府県名","各地の感染者数_1日ごとの発表数","各地の死者数_1日ごとの発表数","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均","新規感染者数先週比"]]
latest.head()

,日付,都道府県名,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
519,2021-06-18,北海道,76,3,605.0,0.041096,-0.074526,-0.475862
1039,2021-06-18,青森県,0,0,8.0,NaN,NaN,-1.000000
1559,2021-06-18,岩手県,5,0,61.0,0.250000,-0.088379,-0.687500
2079,2021-06-18,宮城県,4,1,50.0,-0.500000,0.062168,-0.500000
2599,2021-06-18,秋田県,0,0,3.0,NaN,NaN,NaN


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
0,三重県,12480,5177,109,2021-06-18,14,0,63.0,2.500000,0.334807,-0.066667
1,京都府,13520,16433,235,2021-06-18,18,1,143.0,0.058824,0.047296,-0.379310
2,佐賀県,21320,2543,24,2021-06-18,0,1,6.0,-1.000000,NaN,-1.000000
3,兵庫県,14560,40683,1271,2021-06-18,37,1,235.0,-0.051282,0.052774,-0.195652
4,北海道,520,40870,1334,2021-06-18,76,3,605.0,0.041096,-0.074526,-0.475862


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口
0,三重県,12480,5177,109,2021-06-18,14,0,63.0,2.500000,0.334807,-0.066667,1813859
1,京都府,13520,16433,235,2021-06-18,18,1,143.0,0.058824,0.047296,-0.379310,2545899
2,佐賀県,21320,2543,24,2021-06-18,0,1,6.0,-1.000000,NaN,-1.000000,823810
3,兵庫県,14560,40683,1271,2021-06-18,37,1,235.0,-0.051282,0.052774,-0.195652,5549568
4,北海道,520,40870,1334,2021-06-18,76,3,605.0,0.041096,-0.074526,-0.475862,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,520,40870,1334,2021-06-18,76,3,605.0,0.041096,-0.074526,-0.475862,5267762,77.585130,775.851301,11.484953
41,青森県,1040,2452,31,2021-06-18,0,0,8.0,NaN,NaN,-1.000000,1275783,19.219569,192.195695,0.627066
19,岩手県,1560,1625,46,2021-06-18,5,0,61.0,0.250000,-0.088379,-0.687500,1235517,13.152389,131.523888,4.937204
11,宮城県,2080,9061,86,2021-06-18,4,1,50.0,-0.500000,0.062168,-0.500000,2292385,39.526519,395.265193,2.181134
36,秋田県,2600,764,20,2021-06-18,0,0,3.0,NaN,NaN,NaN,985416,7.753071,77.530708,0.304440
15,山形県,3120,2017,46,2021-06-18,0,0,13.0,NaN,NaN,-1.000000,1082296,18.636307,186.363065,1.201150
35,福島県,3640,4754,159,2021-06-18,13,1,56.0,0.444444,0.664807,0.444444,1881981,25.260616,252.606163,2.975588
38,茨城県,4160,10221,161,2021-06-18,21,1,194.0,0.400000,0.603032,0.400000,2921436,34.986219,349.862191,6.640570
27,栃木県,4680,6762,81,2021-06-18,23,1,133.0,1.090909,0.189383,-0.115385,1965516,34.403180,344.031796,6.766671
37,群馬県,5200,7996,149,2021-06-18,2,0,26.0,0.000000,-0.119048,-0.833333,1969439,40.600394,406.003943,1.320173


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      16579.042553
std       30371.643826
min         466.000000
25%        2497.500000
50%        4954.000000
75%       10784.000000
max      168321.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,6760,168321,2192,2021-06-18,453,2,2723.0,0.002212,0.059360,0.041379,13834925,121.663833,1216.638327,19.682073
1,大阪府,14040,102223,2584,2021-06-18,79,17,671.0,-0.168421,-0.011599,-0.410448,8849635,115.510979,1155.109787,7.582234
2,神奈川県,7280,65120,936,2021-06-18,231,4,1344.0,0.248649,0.031036,0.050000,9209442,70.710039,707.100387,14.593718
3,愛知県,11960,50340,923,2021-06-18,106,6,684.0,0.070707,0.047049,-0.288591,7575530,66.450796,664.507962,9.029071
4,埼玉県,5720,45377,826,2021-06-18,64,1,488.0,-0.179487,0.005023,-0.264368,7390054,61.402799,614.027990,6.603470
5,北海道,520,40870,1334,2021-06-18,76,3,605.0,0.041096,-0.074526,-0.475862,5267762,77.585130,775.851301,11.484953
6,兵庫県,14560,40683,1271,2021-06-18,37,1,235.0,-0.051282,0.052774,-0.195652,5549568,73.308409,733.084089,4.234564
7,千葉県,6240,38936,700,2021-06-18,135,1,733.0,0.088710,0.049457,0.205357,6319772,61.609818,616.098176,11.598520
8,福岡県,20800,35138,510,2021-06-18,26,3,248.0,-0.365854,-0.065867,-0.551724,5129841,68.497250,684.972497,4.834458
9,沖縄県,24440,19967,167,2021-06-18,86,0,720.0,-0.113402,0.007248,-0.406897,1481547,134.771290,1347.712897,48.597851


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      39.980323
std       28.973258
min        7.753071
25%       22.245008
50%       30.185133
75%       52.104216
max      134.771290
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,24440,19967,167,2021-06-18,86,0,720.0,-0.113402,0.007248,-0.406897,1481547,134.771290,1347.712897,48.597851
26,東京都,6760,168321,2192,2021-06-18,453,2,2723.0,0.002212,0.059360,0.041379,13834925,121.663833,1216.638327,19.682073
32,神奈川県,7280,65120,936,2021-06-18,231,4,1344.0,0.248649,0.031036,0.050000,9209442,70.710039,707.100387,14.593718
16,山梨県,9880,1916,20,2021-06-18,35,0,99.0,1.692308,0.291475,-0.125000,826579,23.179878,231.798775,11.977077
5,千葉県,6240,38936,700,2021-06-18,135,1,733.0,0.088710,0.049457,0.205357,6319772,61.609818,616.098176,11.598520
4,北海道,520,40870,1334,2021-06-18,76,3,605.0,0.041096,-0.074526,-0.475862,5267762,77.585130,775.851301,11.484953
24,愛知県,11960,50340,923,2021-06-18,106,6,684.0,0.070707,0.047049,-0.288591,7575530,66.450796,664.507962,9.029071
44,高知県,20280,1705,23,2021-06-18,6,0,60.0,-0.400000,0.162245,-0.250000,709230,24.040156,240.401562,8.459879
9,大阪府,14040,102223,2584,2021-06-18,79,17,671.0,-0.168421,-0.011599,-0.410448,8849635,115.510979,1155.109787,7.582234
27,栃木県,4680,6762,81,2021-06-18,23,1,133.0,1.090909,0.189383,-0.115385,1965516,34.403180,344.031796,6.766671


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    33.000000
mean      0.192522
std       0.246206
min      -0.119048
25%       0.031036
50%       0.103611
75%       0.334807
max       0.794805
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
13,富山県,8320,2000,37,2021-06-18,15,0,51.0,0.363636,0.794805,14.000000,1055999,18.939412,189.394119,4.829550
31,石川県,8840,3901,114,2021-06-18,7,0,33.0,0.000000,0.665646,0.000000,1139612,34.230949,342.309488,2.895722
35,福島県,3640,4754,159,2021-06-18,13,1,56.0,0.444444,0.664807,0.444444,1881981,25.260616,252.606163,2.975588
38,茨城県,4160,10221,161,2021-06-18,21,1,194.0,0.400000,0.603032,0.400000,2921436,34.986219,349.862191,6.640570
10,奈良県,15080,8008,123,2021-06-18,14,1,76.0,0.076923,0.560195,-0.066667,1353837,59.150400,591.503999,5.613674
43,香川県,19240,2081,29,2021-06-18,3,0,20.0,2.000000,0.510204,-0.250000,981280,21.206995,212.069949,2.038154
18,岡山県,17160,7571,123,2021-06-18,3,0,38.0,-0.250000,0.354113,-0.750000,1903627,39.771447,397.714468,1.996189
39,長崎県,21840,3098,69,2021-06-18,11,0,59.0,0.100000,0.344218,0.222222,1350769,22.935084,229.350836,4.367882
0,三重県,12480,5177,109,2021-06-18,14,0,63.0,2.500000,0.334807,-0.066667,1813859,28.541359,285.413585,3.473258
46,鹿児島県,23920,3630,33,2021-06-18,9,0,76.0,2.000000,0.293465,0.125000,1630146,22.267944,222.679441,4.662159


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,日付,各地の感染者数_1日ごとの発表数,各地の死者数_1日ごとの発表数,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
37,群馬県,5200,7996,149,2021-06-18,2,0,26.0,0.000000,-0.119048,-0.833333,1969439,40.600394,406.003943,1.320173
19,岩手県,1560,1625,46,2021-06-18,5,0,61.0,0.250000,-0.088379,-0.687500,1235517,13.152389,131.523888,4.937204
4,北海道,520,40870,1334,2021-06-18,76,3,605.0,0.041096,-0.074526,-0.475862,5267762,77.585130,775.851301,11.484953
34,福岡県,20800,35138,510,2021-06-18,26,3,248.0,-0.365854,-0.065867,-0.551724,5129841,68.497250,684.972497,4.834458
9,大阪府,14040,102223,2584,2021-06-18,79,17,671.0,-0.168421,-0.011599,-0.410448,8849635,115.510979,1155.109787,7.582234
7,埼玉県,5720,45377,826,2021-06-18,64,1,488.0,-0.179487,0.005023,-0.264368,7390054,61.402799,614.027990,6.603470
28,沖縄県,24440,19967,167,2021-06-18,86,0,720.0,-0.113402,0.007248,-0.406897,1481547,134.771290,1347.712897,48.597851
29,滋賀県,13000,5474,88,2021-06-18,4,0,89.0,-0.809524,0.016976,-0.666667,1420948,38.523577,385.235772,6.263424
32,神奈川県,7280,65120,936,2021-06-18,231,4,1344.0,0.248649,0.031036,0.050000,9209442,70.710039,707.100387,14.593718
24,愛知県,11960,50340,923,2021-06-18,106,6,684.0,0.070707,0.047049,-0.288591,7575530,66.450796,664.507962,9.029071


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,5))
plt.subplot(1,2,1) # (rows, columns, panel number)
plt.title("各地の新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")

plt.subplot(1,2,2) # (rows, columns, panel number)
plt.title("1万人当たりの新規感染者数累計")
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [22]:
fig = plt.figure(figsize=(9.5,10))

### 最新の各地の新規感染者数
plt.subplot(2,2,1) # (rows, columns, panel number)
plt.title("{}の各地の新規感染者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の感染者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の感染者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 最新の各地の死者数
plt.subplot(2,2,2) # (rows, columns, panel number)
plt.title("{}の各地の新規死者数".format(latest_date))
chart_data = all_jp.sort_values(["各地の死者数_1日ごとの発表数"])
plt.barh(chart_data["都道府県名"], chart_data["各地の死者数_1日ごとの発表数"])
plt.yticks(fontsize=8)
plt.grid(axis="x")



### 直近1週間の10万人当たりの新規感染者数
plt.subplot(2,2,3) # (rows, columns, panel number)
plt.title("直近1週間の10万人当たりの新規感染者数")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.plot([15,15],[0,47],"--",lw=0.5,color="black")
plt.text(15,0,"Stage3",ha="center",va="top",fontsize=8)
plt.plot([25,25],[0,47],"--",lw=0.5,color="black")
plt.text(25,0,"Stage4",ha="center",va="top",fontsize=8)
plt.yticks(fontsize=8)
plt.grid(axis="x")

### 新規感染者数の増加速度（新規感染者数前週同曜日比）
plt.subplot(2,2,4) # (rows, columns, panel number)
plt.title("新規感染者数前週同曜日比")
chart_data = all_jp.sort_values("新規感染者数先週比",na_position="first")
plt.barh(chart_data["都道府県名"], chart_data["新規感染者数先週比"])
plt.yticks(fontsize=8)
plt.grid(axis="x")

fig.text(0,0,"※新型コロナ関連の情報提供:NHK ※人口の情報提供:e-Stat",fontsize=8)
fig.text(0,0.98,"※{}時点".format(latest_date),fontsize=9,va="top")

fig.tight_layout()

<IPython.core.display.Javascript object>

In [23]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [24]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   466.     21447.875  42429.75   63411.625  84393.5   105375.375
 126357.25  147339.125 168321.   ]


## ヒストグラム

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([38.,  2.,  5.,  0.,  1.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [26]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([ 9., 23.,  4.,  8.,  0.,  1.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)

In [27]:
pref[pref["都道府県名"]=="和歌山県"].tail(10)


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,新規感染者数先週比
15590,2021-06-09,30,和歌山県,2,2641,2,47,12.0,1.000000,NaN,-0.750000
15591,2021-06-10,30,和歌山県,6,2647,0,47,14.0,2.000000,NaN,0.500000
15592,2021-06-11,30,和歌山県,1,2648,0,47,15.0,-0.833333,NaN,inf
15593,2021-06-12,30,和歌山県,3,2651,0,47,16.0,2.000000,0.523810,0.500000
15594,2021-06-13,30,和歌山県,1,2652,0,47,15.0,-0.666667,0.428571,-0.500000
15595,2021-06-14,30,和歌山県,0,2652,1,48,14.0,-1.000000,0.357143,-1.000000
15596,2021-06-15,30,和歌山県,2,2654,0,48,15.0,inf,NaN,1.000000
15597,2021-06-16,30,和歌山県,0,2654,0,48,13.0,-1.000000,NaN,-1.000000
15598,2021-06-17,30,和歌山県,1,2655,0,48,8.0,inf,NaN,-0.833333
15599,2021-06-18,30,和歌山県,1,2656,0,48,8.0,0.000000,NaN,0.000000
